In [83]:
import xml.etree.ElementTree as ET
import numpy as np
import copy

In [4]:
file_path = "data/sample_haifu.xml"

In [98]:
class GameInfo():
    def __init__(self, go: int =161):
        self.go = go

    def against_human(self):
        return self.go & 1
    
    def no_red(self):
        return (self.go & 0x2) >> 1
    
    def kansaki(self):
        return (self.go & 0x4) >> 2
    
    def tonnan(self):
        return (self.go & 0x8) >> 3
    
    def three_players(self):
        return (self.go & 0x10) >> 4
    
    def fast(self):
        return (self.go & 0x40) >> 6
    
    def level(self):
        return (self.go & 0xA0) >> 5

    def __str__(self):
        return f"GameInfo: against_human={self.against_human()}, no_red={self.no_red()}, kansaki={self.kansaki()}, tonnan={self.tonnan()}, three_players={self.three_players()}, fast={self.fast()}, level={self.level()}"

In [99]:
with open(file_path, "rb") as file:
    tree = ET.parse(file)
    root = tree.getroot()
    # xml_str = ET.tostring(root, encoding="utf-8", method="xml") # code to convert xml to string
    # root = ET.fromstring(xml_data) # code to convert string to xml


events = []

# Define a recursive function to traverse the XML elements
def traverse(element):
    item = {"event": element.tag, "attr": element.attrib}
    events.append(item)
    for child in element:
        traverse(child)

traverse(root)

gametype = root.findall("GO")
if len(gametype) != 1:
    raise ValueError("Invalid number of game type elements")
try:
    gameinfo = GameInfo(int(gametype[0].attrib["type"]))
except:
    raise ValueError("Invalid game type element")

# print(events)
print(gameinfo)

GameInfo: against_human=1, no_red=0, kansaki=0, tonnan=1, three_players=0, fast=0, level=5


In [100]:
# sanity check
assert gameinfo.against_human() == 1
assert gameinfo.no_red() == 0
assert gameinfo.kansaki() == 0
assert gameinfo.three_players() == 0

# group events into each round
rounds = []
curr_round_events = []

for event in events:
    if event["event"] in ["mjloggm", "SHUFFLE", "UN", "GO", "TAIKYOKU"]:
        continue

    curr_round_events.append(event)

    if event["event"] in ["AGARI", "RYUUKYOKU"]:
        rounds.append(curr_round_events)
        curr_round = []

rounds

[[{'event': 'INIT',
   'attr': {'seed': '0,0,0,4,3,128',
    'ten': '250,250,250,250',
    'oya': '0',
    'hai0': '19,23,42,59,72,11,67,117,8,122,49,88,75',
    'hai1': '36,98,123,118,106,31,114,55,2,17,61,34,94',
    'hai2': '60,10,97,51,86,30,108,38,121,35,57,47,26',
    'hai3': '126,44,15,43,92,95,127,18,107,56,129,101,80'}},
  {'event': 'T70', 'attr': {}},
  {'event': 'D117', 'attr': {}},
  {'event': 'U84', 'attr': {}},
  {'event': 'E36', 'attr': {}},
  {'event': 'V77', 'attr': {}},
  {'event': 'F108', 'attr': {}},
  {'event': 'W96', 'attr': {}},
  {'event': 'G129', 'attr': {}},
  {'event': 'T9', 'attr': {}},
  {'event': 'D122', 'attr': {}},
  {'event': 'U24', 'attr': {}},
  {'event': 'E2', 'attr': {}},
  {'event': 'V135', 'attr': {}},
  {'event': 'F38', 'attr': {}},
  {'event': 'W54', 'attr': {}},
  {'event': 'G80', 'attr': {}},
  {'event': 'T5', 'attr': {}},
  {'event': 'D70', 'attr': {}},
  {'event': 'U21', 'attr': {}},
  {'event': 'E118', 'attr': {}},
  {'event': 'V53', 'attr'

In [96]:
# build traninig data from player 0's perspective
PLAYER_ID = 0

# constants
N_ROUNDS = 4 * (gameinfo.tonnan() + 1)

kyoku_info = rounds[0]

# sanity check
assert kyoku_info[0]["event"] == "INIT"

scores = kyoku_info[0]["attr"]["ten"].split(",")
scores = [int(score) for score in scores]
parent = kyoku_info[0]["attr"]["oya"]
hands = [kyoku_info[0]["attr"][f'hai{player}'] for player in range(4)]
hands = [list(map(int, hand.split(","))) for hand in hands]
curr_round, honba, kyotaku, _, _, dora = kyoku_info[0]["attr"]["seed"].split(",")
doras = [int(dora)]
remaining_rounds = N_ROUNDS - int(curr_round)
parent_rounds_remaining = [(remaining_rounds + i) // 4 for i in range(4)]

In [97]:
print(f"Scores: {scores}")
print(f"Parent: {parent}")
print(f"Hands: {hands}")
print(f"Current round: {curr_round}")
print(f"Honba: {honba}")
print(f"Kyotaku: {kyotaku}")
print(f"Dora: {doras}")
print(f"Remaining rounds: {remaining_rounds}")
print(f"Parent rounds remaining: {parent_rounds_remaining}")

Scores: [250, 250, 250, 250]
Parent: 0
Hands: [[19, 23, 42, 59, 72, 11, 67, 117, 8, 122, 49, 88, 75], [36, 98, 123, 118, 106, 31, 114, 55, 2, 17, 61, 34, 94], [60, 10, 97, 51, 86, 30, 108, 38, 121, 35, 57, 47, 26], [126, 44, 15, 43, 92, 95, 127, 18, 107, 56, 129, 101, 80]]
Current round: 0
Honba: 0
Kyotaku: 0
Dora: [128]
Remaining rounds: 8
Parent rounds remaining: [2, 2, 2, 2]


In [90]:
REMAINING_TILES = [4] * 34 + [1] * 3
for i in [5, 14, 23]:
    REMAINING_TILES[i] = 3

print(REMAINING_TILES)

[4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1]


In [85]:
TILE2IDX = [i // 4 for i in range(136)]
TILE2IDX[16] = 34
TILE2IDX[52] = 35
TILE2IDX[88] = 36

print(TILE2IDX)

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 34, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 35, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 21, 36, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 33, 33, 33, 33]


In [95]:
REMAINING_TSUMO = 70

In [103]:
class Draw:
    pass

In [104]:
class Naki(Draw):
    def __init__(self, naki_code: int):
        self.naki_code = naki_code
    
    def from_who(self):
        return self.naki_code & 3

    def is_chi(self):
        return (self.naki_code & 0x4) >> 2

    def is_pon(self):
        return not self.is_chi() and (self.naki_code & 0x8)

    def is_kakan(self):
        return not self.is_chi() and (self.naki_code & 0x10)
    
    def is_minkan(self):
        return self.naki_code & 0b111100 == 0 and self.from_who()
    
    def is_ankan(self):
        return self.naki_code & 0b111100 == 0 and not self.from_who()

    def pattern_chi(self):
        pattern = (self.naki_code & 0xFC00) >> 10
        which = pattern % 3
        pattern //= 3
        color = pattern // 7
        number = pattern % 7
        has_red = False
        if 2 <= number <= 4:
            tag = (self.naki_code >> ((6 - number) * 2)) & 3
            if tag == 0:
                has_red = True
        return (color, number, which, has_red)
    
    def pattern_pon(self):
        pattern = (self.naki_code & 0xFE00) >> 9
        which = pattern % 3
        pattern //= 3
        color = pattern // 9
        number = pattern % 9
        has_red = which != 0
        return (color, number, which, has_red)
    
    def pattern_kakan(self):
        pattern = (self.naki_code & 0xFE00) >> 9
        which = pattern % 3
        pattern //= 3
        color = pattern // 9
        number = pattern % 9
        has_red = self.number == 5 and color != 3 and which == 0
        return (color, number, which, has_red)

    def pattern_minkan(self):
        pattern = (self.naki_code & 0xFF00) >> 8
        which = pattern % 4
        pattern //= 4
        color = pattern // 9
        number = pattern % 9
        has_red = self.number == 5 and color != 3 and which == 0
        return (color, number, which, has_red)
    
    def pattern_ankan(self):
        pattern = (self.naki_code & 0xFF00) >> 8
        which = pattern % 4
        pattern //= 4
        color = pattern // 9
        number = pattern % 9
        has_red = self.number == 5 and color != 3 and which == 0
        return (color, number, which, has_red)

In [105]:
class Tsumo(Draw):
    def __init__(self, tile: int):
        self.tile = tile # 0-136

In [106]:
class Discard:
    def __init__(self, tile: int):
        self.tile = tile # 0-136

In [107]:
class StateObject:
    pass

In [111]:
class Action:
    TSUMO = 0
    NAKI = 1

class TsumoAction(Action):
    def __init__(self, draw: Draw = None, stateObj: StateObject = None, discard: Discard = None):
        self.type = Action.TSUMO
        self.draw = draw
        self.stateObj = stateObj
        self.discard = discard

class NakiAction(Action):
    def __init__(self, naki: Naki, stateObj: StateObject = None, discard: Discard = None):
        self.type = Action.NAKI
        self.naki = naki
        self.stateObj = stateObj
        self.discard = discard

In [112]:
class Decision:
    NAKI = 1
    REACH = 2
    AGARI = 3

class NakiDecision(Decision):
    def __init__(self, naki: Naki, executed: bool):
        self.naki = naki
        self.executed = executed
    
class ReachDecision(Decision):
    def __init__(self, executed: bool):
        self.executed = executed

class AgariDecision(Decision):
    def __init__(self, executed: bool):
        self.executed = executed

class PassDecision(Decision):
    def __init__(self, executed: bool):
        self.executed = executed

In [93]:
remaining_tiles = REMAINING_TILES.copy()
remaining_tsumo = REMAINING_TSUMO
reaches = []
processed = []

for hand in hands:
    for tile in hand:
        remaining_tiles[TILE2IDX[int(tile)]] -= 1

for event in kyoku_info[1:-1]:
    eventtype = event["event"]
    if eventtype == "DORA":
        doras.append(int(event["attr"]["hai"]))
        remaining_tiles[TILE2IDX[int(event["attr"]["hai"])]] -= 1
        remaining_tsumo -= 1
    elif eventtype == "REACH":
        if event["attr"]["step"] == "1":
            continue
        reaches.append(int(event["attr"]["who"]))
        kyotaku += 1
    elif eventtype == "N":
        naki = Naki(int(event["attr"]["m"]))
        processed.append(naki)
        if not naki.is_chi() and not naki.is_pon():
            remaining_tsumo -= 1
    elif eventtype.startswith("T"):
        tile = TILE2IDX[int(event[1:])]
        remaining_tiles[tile] -= 1
        remaining_tsumo -= 1
        